<div style="text-align: justify;font-size: 36px;color: #b1c5fc;">Pre processing flow</div>

<div style="text-align: justify;font-size: 28px;color: #b1c5fc;">Imports and config</div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import nibabel as nib
from nilearn import image
from nilearn.image import resample_to_img
import fsl.wrappers.bet as bet

In [2]:
#Setup FSL environment (for brain extraction)

# Ensure FSLDIR is set
os.environ['FSLDIR'] = '/Users/imeag/fsl'
os.environ['PATH'] = os.environ['FSLDIR'] + '/bin:' + os.environ['PATH']

# Set FSLOUTPUTTYPE environment variable
#os.environ['FSLOUTPUTTYPE'] = 'NIFTI_GZ'
os.environ['FSLOUTPUTTYPE'] = 'NIFTI'


<div style="text-align: justify;font-size: 28px;color: #b1c5fc;">Define functions</div>

<div style="text-align: justify;font-size: 20px;color: #b1c5fc;">Utils</div>

In [3]:
def cargar_nii(archivo):
    """
    Carga un archivo .nii (NIfTI) y devuelve la imagen como un array de NumPy.
    
    :param archivo: Ruta al archivo .nii
    :return: Imagen cargada como un array de NumPy
    """
    imagen_nifti = nib.load(archivo)
    imagen_array = imagen_nifti.get_fdata()
    return imagen_array


<div style="text-align: justify;font-size: 20px;color: #b1c5fc;">Co-registration</div>

In [4]:
def reorient_image(reference_image_path, image_to_reorient_path, output_path):
    """
    Reorienta una imagen a la orientación de una imagen de referencia y guarda el resultado.

    Parameters:
    - reference_image_path (str): Ruta del archivo .nii de referencia.
    - image_to_reorient_path (str): Ruta del archivo .nii que se quiere reorientar.
    - output_path (str): Ruta donde se guardará la imagen reorientada.
    """
    # Verificar si los archivos existen
    if not (os.path.exists(reference_image_path) and os.path.exists(image_to_reorient_path)):
        raise FileNotFoundError("Uno o ambos archivos no se encontraron.")

    try:
        # Cargar las imágenes
        imagen1 = nib.load(image_to_reorient_path)  # Imagen a reorientar
        imagen2 = nib.load(reference_image_path)  # Imagen de referencia
        
        # Comprobar los tipos de datos
        print("Tipo de imagen a reorientar:", type(imagen1))  # Debería mostrar <class 'nibabel.nifti1.Nifti1Image'>
        print("Tipo de imagen de referencia:", type(imagen2))  # Debería mostrar <class 'nibabel.nifti1.Nifti1Image'>
        
        # Mostrar información básica de las imágenes
        print("Imagen a reorientar:")
        print("Tipo de datos:", imagen1.get_data_dtype())
        print("Forma:", imagen1.shape)
        
        print("\nImagen de referencia:")
        print("Tipo de datos:", imagen2.get_data_dtype())
        print("Forma:", imagen2.shape)

        # Reorientar la imagen
        reoriented_img = resample_to_img(imagen1, imagen2, interpolation='nearest')

        # Guardar la imagen reorientada
        reoriented_img.to_filename(output_path)

        print(f"Reorientación completa y imagen guardada como '{output_path}'.")

    except Exception as e:
        print(f"Error al cargar las imágenes o realizar la reorientación: {e}")

<div style="text-align: justify;font-size: 20px;color: #b1c5fc;">Normalization</div>

In [5]:
def normalize_image(image_path, output_path):
    """
    Normaliza la escala de grises de una imagen y guarda el resultado.

    Parameters:
    - image_path (str): Ruta del archivo .nii de la imagen.
    - output_path (str): Ruta donde se guardará la imagen normalizada.
    """
    # Cargar la imagen
    img = nib.load(image_path)
    data = img.get_fdata()

    # Normalizar la escala de grises
    min_val = np.min(data)
    max_val = np.max(data)
    normalized_data = (data - min_val) / (max_val - min_val)

    # Crear una nueva imagen con los datos normalizados
    normalized_img = nib.Nifti1Image(normalized_data, img.affine, img.header)
    
    # Guardar la imagen normalizada
    normalized_img.to_filename(output_path)

    print(f"Imagen normalizada guardada como '{output_path}'.")

<div style="text-align: justify;font-size: 20px;color: #b1c5fc;">Brain Extraction</div>

In [ ]:
def extract_brain(input_path, output_path):
    """
    Extrae el cerebro de una imagen y guarda el resultado.

    Parameters:
    - input_path (str): Ruta del archivo .nii de la imagen.
    - output_path (str): Ruta donde se guardará la imagen con el cerebro extraído.
    """
    # Extraer el cerebro
    result = bet(input_path, output_path, f=0.32)

    print(f"Extracción de cerebro completa y resultado guardado como '{output_path}'.")